# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json
import time


# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tazovsky,99,RU,1600183136,84,67.47,78.70,53.76,9.10
1,Olinda,40,BR,1600183137,69,-8.01,-34.86,82.40,12.75
2,Port Alfred,100,ZA,1600183137,68,-33.59,26.89,60.01,15.61
3,Abong Mbang,61,CM,1600183137,80,3.98,13.18,79.45,2.59
4,Lompoc,90,US,1600182934,100,34.64,-120.46,53.60,4.70
...,...,...,...,...,...,...,...,...,...
537,Butka,100,RU,1600183263,95,56.72,63.79,50.97,8.43
538,Prince Albert,90,CA,1600183264,81,53.20,-105.77,48.20,2.24
539,Mananjary,41,MG,1600183264,81,-21.22,48.33,69.62,10.67
540,Agadez,0,NE,1600183264,8,19.75,10.25,104.50,6.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [17]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
43,Guerrero Negro,0,MX,1600183145,51,27.98,-114.06,78.04,8.25
70,San Quintín,0,MX,1600183117,53,30.48,-115.95,76.96,8.32
71,Mount Isa,0,AU,1600183152,27,-20.73,139.50,73.40,5.82
117,Divnomorskoye,0,RU,1600183162,49,44.50,38.14,75.27,9.86
189,Dawlatabad,0,AF,1600183181,22,36.41,64.91,70.48,5.57
230,Yulara,0,AU,1600183191,26,-25.24,130.99,71.60,5.82
266,Dustlik,0,UZ,1600183199,21,40.52,68.04,72.82,5.95
362,Romitan,0,UZ,1600183223,29,39.93,64.38,77.00,5.88
379,Rajaori,0,IN,1600183226,48,33.38,74.30,73.92,5.97
454,Mashivka,0,UA,1600183244,51,49.44,34.87,73.74,9.46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Guerrero Negro,MX,27.98,-114.06,
70,San Quintín,MX,30.48,-115.95,
71,Mount Isa,AU,-20.73,139.50,
117,Divnomorskoye,RU,44.50,38.14,
189,Dawlatabad,AF,36.41,64.91,
230,Yulara,AU,-25.24,130.99,
266,Dustlik,UZ,40.52,68.04,
362,Romitan,UZ,39.93,64.38,
379,Rajaori,IN,33.38,74.30,
454,Mashivka,UA,49.44,34.87,


In [20]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [21]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")


Retrieving Results for Index 43: Guerrero Negro.
Closest hotel in Guerrero Negro is Hotel Los Caracoles.
------------
Retrieving Results for Index 70: San Quintín.
Closest hotel in San Quintín is Hotel Jardines Baja.
------------
Retrieving Results for Index 71: Mount Isa.
Closest hotel in Mount Isa is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 117: Divnomorskoye.
Closest hotel in Divnomorskoye is Park House.
------------
Retrieving Results for Index 189: Dawlatabad.
Missing field/result... skipping.
------------
Retrieving Results for Index 230: Yulara.
Closest hotel in Yulara is The Lost Camel Hotel - Ayers Rock Resort.
------------
Retrieving Results for Index 266: Dustlik.
Missing field/result... skipping.
------------
Retrieving Results for Index 362: Romitan.
Missing field/result... skipping.
------------
Retrieving Results for Index 379: Rajaori.
Closest hotel in Rajaori is Hotel Tariq Residency Rajouri.
------------
Retrieving Results for Index 454: Ma

In [22]:
hotel_df


,City,Country,Lat,Lng,Hotel Name
43,Guerrero Negro,MX,27.98,-114.06,Hotel Los Caracoles
70,San Quintín,MX,30.48,-115.95,Hotel Jardines Baja
71,Mount Isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona
117,Divnomorskoye,RU,44.50,38.14,Park House
189,Dawlatabad,AF,36.41,64.91,
230,Yulara,AU,-25.24,130.99,The Lost Camel Hotel - Ayers Rock Resort
266,Dustlik,UZ,40.52,68.04,
362,Romitan,UZ,39.93,64.38,
379,Rajaori,IN,33.38,74.30,Hotel Tariq Residency Rajouri
454,Mashivka,UA,49.44,34.87,


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))